In [112]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
from collections import Counter
import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df = pd.read_csv('DIPS_Data.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df = df.dropna(how='all')

In [113]:
#assigns values to each row in the column and removes blank rows
data = df.copy()
data = data.fillna({'case_name':'none'})
data['Case'] = 'xxxx'
value = 'Fees:'
case = 'xxxx'
for row in data.itertuples():
    if row[1] not in ['none', 'Fees:', 'Agents:','Lender Groups:', 'Roll ups:', 'Lenders:']:
        case = row[1]
        data.at[row[0], 'Case'] = case
        continue
    if row[1] == 'Fees:':
        value = 'Fees:'
        continue
    if row[1] == 'Agents:':
        value = 'Agents:'
        continue
    if row[1] == 'Lender Groups:':
        value = 'Lender Groups:'
        continue
    if row[1] == 'Roll ups:':
        value = 'Roll ups:'
        continue
    if row[1] == 'Lenders:':
        value = 'Lenders:'
        continue
    data.at[row[0], 'Case'] = case
    data.at[row[0], 'case_name'] = value
data = data[~data.Case.isin(['xxxx'])]
data = data.reset_index(drop=True)
#data = data.set_index('Case')

In [114]:
starting_list = list(data['case_name'])
clean_list = [x for x in starting_list if str(x) != 'nan']
unique_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
all_cases = [x for x in clean_list if str(x) not in ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']]
total_cases = len(all_cases)
#counts the total number of cases present in the dataframe
#observation_count = Counter(clean_list) #dictionary of count of unique deals, Fees, Agents ect.

#combs through the created list and returns the case names that have missing data i.e No Agents, No Lenders ect.
master_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
case_dict = {}
case_index = []
agent_no_lenders = []
no_data = []
lenders_no_agent = []
one_lender_no_agent = []
no_agent = []

for i in range(0, len(clean_list)):
    if clean_list[i] not in master_list:
        name = clean_list[i]
        case_index.append(i)
    
for i in range(1, len(case_index)):
    case_line = clean_list[case_index[i-1]:case_index[i]]
    num_fees = case_line.count('Fees:')
    num_agents = case_line.count('Agents:')
    num_lender_groups = case_line.count('Lender_Groups:')
    num_lenders = case_line.count('Lenders:')
    num_roll_ups = case_line.count('Roll_ups:')
    
    dup_case = case_line[0] in case_dict
    #Creates a Dictionary for all of the cases and the number of DIP Financing they have; as well as the number of Agents, Fees, Lenders, Lender Groups ect.
    #Case Name -> [Case_Count, Dip_Number_X] -> [Agent_Number, Fees_Number, Lender_Groups_Number, Lenders_Number, Roll_Ups_Number]
    if not dup_case:
        case_dict[case_line[0]] = {}
        case_dict[case_line[0]]['Case_Count'] = 1
        case_dict[case_line[0]]['Dip_Number_1'] = {}
        case_dict[case_line[0]]['Dip_Number_1']['Count'] = {}
        case_dict[case_line[0]]['Dip_Number_1']['Count']['Agents'] = num_agents
        case_dict[case_line[0]]['Dip_Number_1']['Count']['Fees'] = num_fees
        case_dict[case_line[0]]['Dip_Number_1']['Count']['Lender_Groups'] = num_lender_groups
        case_dict[case_line[0]]['Dip_Number_1']['Count']['Lenders'] = num_lenders
        case_dict[case_line[0]]['Dip_Number_1']['Count']['Roll_Ups'] = num_roll_ups
    if dup_case:
        case_dict[case_line[0]]['Case_Count'] = case_dict[case_line[0]]['Case_Count'] + 1
        dip_num = 'Dip_Number_' + str(case_dict[case_line[0]]['Case_Count'])
        case_dict[case_line[0]][dip_num] = {}
        case_dict[case_line[0]][dip_num]['Count'] = {}
        case_dict[case_line[0]][dip_num]['Count']['Agents'] = num_agents
        case_dict[case_line[0]][dip_num]['Count']['Fees'] = num_fees
        case_dict[case_line[0]][dip_num]['Count']['Lender_Groups'] = num_lender_groups
        case_dict[case_line[0]][dip_num]['Count']['Lenders'] = num_lenders
        case_dict[case_line[0]][dip_num]['Count']['Roll_Ups'] = num_roll_ups
        
    #print(case_line)
    if ('Agents:' in case_line) and (('Lenders:' not in case_line) and ('Lender Groups:' not in case_line)):
        agent_no_lenders.append(case_line[0])
    if ('Agents:' not in case_line):
        no_agent.append(case_line[0])
    if ('Agents:' not in case_line) and ('Lenders:' not in case_line) and ('Lender Groups:' not in case_line):
        no_data.append(case_line[0])
    if ('Agents:' not in case_line) and ('Lenders:' in case_line):
        lenders_no_agent.append(case_line[0])
        if (case_line.count('Lenders:') == 1):
            one_lender_no_agent.append((case_line[0], case_dict[case_line[0]]['Case_Count']))
        
# Adds the Single Lender as the Admin Agent for all of the cases with no Agent
# Updates the case_dict to reflect the new number of agents
added_agents = pd.DataFrame()
for i in one_lender_no_agent:
    case_dict[i[0]]['Dip_Number_'+str(i[1])]['Agent_Number'] = 1
    lender = data[(data['case_name']=='Lenders:') & (data['Case']==i[0])].reset_index(drop=True).at[i[1]-1, 'case_status']
    added_agents = added_agents.append({'Case': i[0], 'case_name':'Agents:', 'case_status': lender, 'petition_date':'Admin Agent'}, ignore_index=True)
    #may not want to append to the main dataframe, may want to create a separate frame and use

In [115]:
added_agents.to_csv('Added_Agents.csv', encoding='utf-8', index=False)
data.to_csv('Cleaned_DIP_Data.csv', encoding='utf-8', index=False)

In [13]:
one_lender_set = set()
for i in one_lender_no_agent:
    one_lender_set.add(i[0])
set(no_agent).difference(set(one_lender_set))

{'In re Abengoa Bioenergy US Holding, LLC',
 'In re Acadiana Management Group, LLC',
 'In re CS Mining, LLC',
 'In re Cosi, Inc.',
 'In re Ginger Oil Company',
 'In re GulfMark Offshore, Inc.',
 'In re Hydrocarb Energy Corporation',
 'In re Katy Industries, Inc.',
 'In re Molycorp Minerals, LLC',
 'In re Mountain Divide, LLC (Mountainview Energy)',
 'In re Oi Brasil Holdings Co&ouml;peratief U.A.',
 'In re Outer Harbor Terminal, LLC (Ports America)',
 'In re Pacific Exploration &amp; Production Corporation',
 'In re The Engy Group, LLC',
 'In re Videology, Inc.'}

In [117]:
data.head(10)

,case_name,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,unnamed:_7,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,unnamed:_15,fixed/floating,pik,amount,floating_type,floating_amount,unnamed:_21,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,unnamed:_28,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period,Case
0,In re Fairway Group Holdings Corp.,confirmed,5/2/2016,6/8/2016,NaN,NaN,7/3/2016,NaN,DIP Term Loan,USD,55000000.0,3.0,Term Loan,NaN,NaN,NaN,Floating,NaN,NaN,LIBOR,8.0,NaN,250000.0,Yes,1000000.0,NaN,30.0,30.0,NaN,Y,NaN,Y,NaN,"Liens,FeesExpenses",NaN,15.0,542000000,13,In re Fairway Group Holdings Corp.
1,Fees:,Unused Commitment Fee,NaN,0.375,%,Monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
2,Fees:,Other Fee,Duration Fee,0.25,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
3,Fees:,Commitment Fee,NaN,2,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
4,Fees:,Arrangement/Origination Fee,NaN,0.5,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
5,Agents:,Credit Suisse,Admin Agent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
6,Agents:,Credit Suisse,Collateral Agent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
7,Lender Groups:,Consenting Lender Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
8,"In re Arch Coal, Inc.",confirmed,1/11/2016,9/15/2016,NaN,10/5/2016,10/5/2016,NaN,DIP Term Loan,USD,275000000.0,12.0,Term Loan,NaN,NaN,NaN,Floating,NaN,NaN,LIBOR,9.0,NaN,250000.0,NaN,10000000.0,NaN,64.0,NaN,NaN,Y,Additional up to $75mm carve-out in favor of c...,N,Collateral package excludes Receivables Rights,"Liens,Interest,FeesExpenses,FinancialInformation",Yes,NaN,NaN,NaN,"In re Arch Coal, Inc."
9,Fees:,Unused Commitment Fee,NaN,5,%,Annually,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In re Arch Coal, Inc."
